## **Crawling**

### Detik

In [6]:
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import os

# --- Konfigurasi ---
os.makedirs("../data", exist_ok=True)
URL_OUTPUT_FILE = "../data/crawled_urls.txt"
LOG_FILE = "../data/crawl_logs.txt"
PORTAL_NAME = "Detik-Inet"

def log_message(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] {message}\n"
    print(log_entry.strip())
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(log_entry)

def extract_links_from_html(html):
    """Ambil semua link artikel game dari halaman indeks Detik."""
    soup = BeautifulSoup(html, "html.parser")
    anchors = soup.select("article.list-content__item a.media__link")
    
    urls = []
    for a in anchors:
        href = a.get("href")
        if href and "/games-news/" in href:
            urls.append(f"{PORTAL_NAME};{href.strip()}")
            
    return list(set(urls))

def crawl_detik(start_date_str="2020-01-01", end_date_str="2025-11-01"):
    log_message(f"===== Memulai Proses Crawling {PORTAL_NAME} =====")
    
    base_url = "https://inet.detik.com/games-news/indeks?date="
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

    total_days = (end_date - start_date).days + 1
    day_counter = 1
    all_urls_found = []

    current_date = start_date
    while current_date <= end_date:
        date_for_log = current_date.strftime('%Y-%m-%d')
        log_message(f"({day_counter}/{total_days}) Memproses tanggal: {date_for_log}")

        date_for_url = current_date.strftime("%m/%d/%Y")
        url = base_url + date_for_url

        try:
            response = requests.get(url, timeout=15)
            response.raise_for_status()
            page_urls = extract_links_from_html(response.text)
            
            if page_urls:
                all_urls_found.extend(page_urls)
                log_message(f"  -> Ditemukan {len(page_urls)} link.")
            else:
                log_message(f"  -> Tidak ada artikel ditemukan.")
        
        except requests.exceptions.RequestException as e:
            log_message(f"  -> ERROR: {e}")
        
        current_date += timedelta(days=1)
        day_counter += 1

    return all_urls_found

if __name__ == "__main__":
    detik_urls = crawl_detik(start_date_str="2020-01-01", end_date_str="2025-11-08")

    if detik_urls:
        unique_urls = sorted(list(set(detik_urls)))
        log_message(f"Menyimpan total {len(unique_urls)} link unik yang ditemukan...")
        with open(URL_OUTPUT_FILE, "a", encoding="utf-8") as f:
            for url in unique_urls:
                f.write(url + "\n")
        log_message("Penyimpanan link berhasil.")
    else:
        log_message("Tidak ada link baru yang ditemukan pada sesi crawling ini.")
    
    log_message(f"===== Proses Crawling Selesai =====\n")

[2025-11-09 02:08:12] ===== Memulai Proses Crawling Detik-Inet =====
[2025-11-09 02:08:12] (1/2139) Memproses tanggal: 2020-01-01
[2025-11-09 02:08:13]   -> Tidak ada artikel ditemukan.
[2025-11-09 02:08:13] (2/2139) Memproses tanggal: 2020-01-02
[2025-11-09 02:08:13]   -> Ditemukan 1 link.
[2025-11-09 02:08:13] (3/2139) Memproses tanggal: 2020-01-03
[2025-11-09 02:08:14]   -> Ditemukan 1 link.
[2025-11-09 02:08:14] (4/2139) Memproses tanggal: 2020-01-04
[2025-11-09 02:08:14]   -> Ditemukan 1 link.
[2025-11-09 02:08:14] (5/2139) Memproses tanggal: 2020-01-05
[2025-11-09 02:08:15]   -> Ditemukan 1 link.
[2025-11-09 02:08:15] (6/2139) Memproses tanggal: 2020-01-06
[2025-11-09 02:08:15]   -> Tidak ada artikel ditemukan.
[2025-11-09 02:08:15] (7/2139) Memproses tanggal: 2020-01-07
[2025-11-09 02:08:16]   -> Ditemukan 4 link.
[2025-11-09 02:08:16] (8/2139) Memproses tanggal: 2020-01-08
[2025-11-09 02:08:16]   -> Tidak ada artikel ditemukan.
[2025-11-09 02:08:16] (9/2139) Memproses tanggal: 

### Gamebrott

In [7]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os
import re 

# --- Konfigurasi ---
SITEMAP_INDEX_URL = "https://gamebrott.com/sitemap_index.xml"
URL_OUTPUT_FILE = "../data/crawled_urls.txt" 
LOG_FILE = "../data/crawl_logs.txt"
PORTAL_NAME = "Gamebrott"

output_dir = os.path.dirname(URL_OUTPUT_FILE)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def log_to_file(message):
    """Fungsi untuk menulis pesan log ke file dengan timestamp."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] {message}\n"
    print(log_entry.strip())
    with open(LOG_FILE, 'a', encoding='utf-8') as f:
        f.write(log_entry)

def get_sitemap_number(url):
    """Mengekstrak nomor dari URL sitemap untuk sorting yang benar."""
    match = re.search(r'post-sitemap(\d+)\.xml', url)
    return int(match.group(1)) if match else 0

def crawl_xml_sitemap():
    """
    Fungsi utama untuk crawling sitemap XML, dengan logging dan penyimpanan append.
    """
    log_to_file(f"===== Memulai Proses Crawling {PORTAL_NAME} =====")
    
    newly_found_urls = []
    
    try:
        log_to_file(f"Mengambil sitemap index dari: {SITEMAP_INDEX_URL}")
        response_main = requests.get(SITEMAP_INDEX_URL, headers=HEADERS, timeout=15)
        response_main.raise_for_status()

        soup_main = BeautifulSoup(response_main.content, "xml")
        all_sitemap_links = [loc.text for loc in soup_main.find_all('loc')]
        
        if not all_sitemap_links:
            log_to_file("ERROR: Tidak ada tag <loc> yang ditemukan di sitemap index.")
            return

        log_to_file(f"Total sitemap ditemukan di index: {len(all_sitemap_links)}")
        
        post_sitemap_links = [url for url in all_sitemap_links if 'post-sitemap' in url]
        log_to_file(f"Menyaring sitemap... Ditemukan {len(post_sitemap_links)} link yang mengandung 'post-sitemap'.")

        post_sitemap_links.sort(key=get_sitemap_number)
        
        sitemaps_to_process = post_sitemap_links[:50]
        log_to_file(f"Membatasi proses hanya untuk {len(sitemaps_to_process)} sitemap pertama.")
        
        for i, sitemap_url in enumerate(sitemaps_to_process, 1):
            log_to_file(f"({i}/{len(sitemaps_to_process)}) Memproses sitemap: {sitemap_url}")
            try:
                response_post = requests.get(sitemap_url, headers=HEADERS, timeout=15)
                response_post.raise_for_status()

                soup_post = BeautifulSoup(response_post.content, "xml")
                
                article_links_in_page = []
                url_blocks = soup_post.find_all('url')
                
                for block in url_blocks:
                    loc_tag = block.find('loc')
                    if loc_tag:
                        article_links_in_page.append(loc_tag.text)
                
                log_to_file(f"  -> Ditemukan {len(article_links_in_page)} link artikel (gambar diabaikan).")
                newly_found_urls.extend(article_links_in_page)

            except requests.exceptions.RequestException as e:
                log_to_file(f"  -> ERROR: Gagal mengambil atau memproses {sitemap_url}: {e}")
                continue

    except requests.exceptions.RequestException as e:
        log_to_file(f"FATAL ERROR: Gagal mengambil sitemap index utama. Proses dihentikan. Error: {e}")
        return

    if newly_found_urls:
        log_to_file(f"Menyimpan {len(newly_found_urls)} link baru ke file {URL_OUTPUT_FILE}...")
        try:
            with open(URL_OUTPUT_FILE, 'a', encoding='utf-8') as f:
                for url in newly_found_urls:
                    # Menggunakan variabel PORTAL_NAME
                    f.write(f"{PORTAL_NAME};{url}\n")
            log_to_file("Penyimpanan link berhasil.")
        except IOError as e:
            log_to_file(f"ERROR: Gagal menulis ke file {URL_OUTPUT_FILE}: {e}")
    else:
        log_to_file("Tidak ada link baru yang ditemukan pada sesi crawling ini.")
    
    log_to_file(f"Total link yang didapat pada sesi ini: {len(newly_found_urls)}")
    log_to_file("===== Proses Crawling Selesai =====\n")


if __name__ == "__main__":
    crawl_xml_sitemap()

[2025-11-09 02:30:59] ===== Memulai Proses Crawling Gamebrott =====
[2025-11-09 02:30:59] Mengambil sitemap index dari: https://gamebrott.com/sitemap_index.xml
[2025-11-09 02:31:00] Total sitemap ditemukan di index: 243
[2025-11-09 02:31:00] Menyaring sitemap... Ditemukan 217 link yang mengandung 'post-sitemap'.
[2025-11-09 02:31:00] Membatasi proses hanya untuk 50 sitemap pertama.
[2025-11-09 02:31:00] (1/50) Memproses sitemap: https://gamebrott.com/post-sitemap1.xml
[2025-11-09 02:31:00]   -> Ditemukan 201 link artikel (gambar diabaikan).
[2025-11-09 02:31:00] (2/50) Memproses sitemap: https://gamebrott.com/post-sitemap2.xml
[2025-11-09 02:31:00]   -> Ditemukan 200 link artikel (gambar diabaikan).
[2025-11-09 02:31:00] (3/50) Memproses sitemap: https://gamebrott.com/post-sitemap3.xml
[2025-11-09 02:31:01]   -> Ditemukan 200 link artikel (gambar diabaikan).
[2025-11-09 02:31:01] (4/50) Memproses sitemap: https://gamebrott.com/post-sitemap4.xml
[2025-11-09 02:31:01]   -> Ditemukan 200 

### Kotakgames

In [8]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

START_YEAR = 2022
END_YEAR = datetime.now().year 

# --- Konfigurasi ---
URL_OUTPUT_FILE = "../data/crawled_urls.txt"
LOG_FILE = "../data/crawl_logs.txt"
PORTAL_NAME = "Kotakgame"

output_dir = os.path.dirname(URL_OUTPUT_FILE)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def log_to_file(message):
    timestamp = datetime.now().strftime("%Y-%-m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] {message}\n"
    print(log_entry.strip())
    with open(LOG_FILE, 'a', encoding='utf-8') as f:
        f.write(log_entry)

def get_last_page(year_base_url):
    """Fungsi ini sekarang menerima base URL untuk tahun tertentu."""
    first_page_url = year_base_url + "1/"
    try:
        log_to_file(f"  -> Mencari halaman terakhir dari: {first_page_url}")
        response = requests.get(first_page_url, headers=HEADERS, timeout=15, verify=False)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, "html.parser")
        last_page_link = soup.find('a', class_='prevnext', text='LAST »')
        
        if last_page_link and last_page_link.has_attr('href'):
            href = last_page_link['href']
            page_number = int(href.strip('/').split('/')[-1])
            log_to_file(f"  -> Halaman terakhir ditemukan: {page_number}")
            return page_number
        else:
            log_to_file("  -> WARNING: Link 'LAST »' tidak ditemukan. Mengasumsikan hanya ada 1 halaman.")
            return 1
            
    except requests.exceptions.RequestException as e:
        log_to_file(f"  -> ERROR: Tidak bisa mengakses halaman pertama untuk tahun ini. Error: {e}")
        return 0 # Mengembalikan 0 jika gagal, untuk dilewati
    except (ValueError, IndexError):
        log_to_file("  -> FATAL ERROR: Gagal mem-parsing nomor halaman terakhir dari link.")
        return 0

def crawl_kotakgame_multi_year():
    log_to_file(f"===== Memulai Proses Crawling {PORTAL_NAME} (Tahun {START_YEAR}-{END_YEAR}) =====")
    
    grand_total_urls = []
    
    for year in range(START_YEAR, END_YEAR + 1):
        log_to_file(f"--- Memproses Tahun {year} ---")
        
        # Base URL sekarang dinamis berdasarkan tahun
        base_url_for_year = f"https://www.kotakgame.com/berita/index/{year}/0/0/"
        
        last_page = get_last_page(base_url_for_year)
        if last_page == 0:
            log_to_file(f"Melewati tahun {year} karena gagal mendapatkan info halaman.")
            continue # Lanjut ke tahun berikutnya

        urls_this_year = []
        for page_num in range(1, last_page + 1):
            page_url = f"{base_url_for_year}{page_num}/"
            log_to_file(f"    ({page_num}/{last_page}) Memproses halaman: {page_url}")
            
            try:
                response = requests.get(page_url, headers=HEADERS, timeout=15, verify=False)
                response.raise_for_status()
                
                soup = BeautifulSoup(response.content, "html.parser")
                article_links_found = soup.select('div#contenta div.detailfeature h4 a')
                
                page_urls = []
                for link_tag in article_links_found:
                    if link_tag.has_attr('href'):
                        relative_url = link_tag['href']
                        full_url = f"https://www.kotakgame.com{relative_url}"
                        page_urls.append(full_url)
                
                log_to_file(f"      -> Ditemukan {len(page_urls)} link artikel.")
                urls_this_year.extend(page_urls)

            except requests.exceptions.RequestException as e:
                log_to_file(f"      -> ERROR: Gagal mengambil {page_url}: {e}")
                continue
        
        log_to_file(f"--- Selesai Tahun {year}, ditemukan {len(urls_this_year)} link ---")
        grand_total_urls.extend(urls_this_year)

    if grand_total_urls:
        unique_urls = sorted(list(set(grand_total_urls)))
        log_to_file(f"Menyimpan total {len(unique_urls)} link unik dari semua tahun ke file {URL_OUTPUT_FILE}...")
        try:
            with open(URL_OUTPUT_FILE, 'a', encoding='utf-8') as f:
                for url in unique_urls:
                    f.write(f"{PORTAL_NAME};{url}\n")
            log_to_file("Penyimpanan link berhasil.")
        except IOError as e:
            log_to_file(f"ERROR: Gagal menulis ke file {URL_OUTPUT_FILE}: {e}")
    else:
        log_to_file("Tidak ada link baru yang ditemukan pada sesi crawling ini.")
    
    log_to_file(f"Total link yang didapat pada sesi ini: {len(grand_total_urls)}")
    log_to_file("===== Proses Crawling Selesai =====\n")

if __name__ == "__main__":
    crawl_kotakgame_multi_year()


[2025-11-09 02:31:18] ===== Memulai Proses Crawling Kotakgame (Tahun 2022-2025) =====
[2025-11-09 02:31:18] --- Memproses Tahun 2022 ---
[2025-11-09 02:31:18]   -> Mencari halaman terakhir dari: https://www.kotakgame.com/berita/index/2022/0/0/1/


/tmp/ipykernel_7555/1082091645.py:41: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  last_page_link = soup.find('a', class_='prevnext', text='LAST »')


[2025-11-09 02:31:19]   -> Halaman terakhir ditemukan: 238
[2025-11-09 02:31:19]     (1/238) Memproses halaman: https://www.kotakgame.com/berita/index/2022/0/0/1/
[2025-11-09 02:31:20]       -> Ditemukan 20 link artikel.
[2025-11-09 02:31:20]     (2/238) Memproses halaman: https://www.kotakgame.com/berita/index/2022/0/0/2/
[2025-11-09 02:31:21]       -> Ditemukan 20 link artikel.
[2025-11-09 02:31:21]     (3/238) Memproses halaman: https://www.kotakgame.com/berita/index/2022/0/0/3/
[2025-11-09 02:31:22]       -> Ditemukan 20 link artikel.
[2025-11-09 02:31:22]     (4/238) Memproses halaman: https://www.kotakgame.com/berita/index/2022/0/0/4/
[2025-11-09 02:31:23]       -> Ditemukan 20 link artikel.
[2025-11-09 02:31:23]     (5/238) Memproses halaman: https://www.kotakgame.com/berita/index/2022/0/0/5/
[2025-11-09 02:31:24]       -> Ditemukan 20 link artikel.
[2025-11-09 02:31:24]     (6/238) Memproses halaman: https://www.kotakgame.com/berita/index/2022/0/0/6/
[2025-11-09 02:31:25]      

### Indogamers

In [9]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os
import time 
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# --- Konfigurasi ---
BASE_URL = "https://indogamers.com/"
CATEGORIES = ['e-sport', 'guides', 'pc', 'console', 'mobile'] 
URL_OUTPUT_FILE = "../data/crawled_urls.txt"
LOG_FILE = "../data/crawl_logs.txt"

output_dir = os.path.dirname(URL_OUTPUT_FILE)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def log_to_file(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] {message}\n"
    print(log_entry.strip())
    with open(LOG_FILE, 'a', encoding='utf-8') as f:
        f.write(log_entry)

def crawl_indogamers_deep():
    log_to_file("===== Memulai Proses Crawling Mendalam Indogamers.com =====")
    
    total_urls_found_session = 0

    for category in CATEGORIES:
        log_to_file(f"Memulai kategori: '{category}'")
        page_num = 1
        urls_per_category = 0
        
        while True:
            page_url = f"{BASE_URL}{category}?page={page_num}"
            log_to_file(f"  -> Memproses halaman: {page_url}")
            
            try:
                response = requests.get(page_url, headers=HEADERS, timeout=20, verify=False)
                
                if response.status_code == 404:
                    log_to_file(f"    -> Halaman tidak ditemukan (404). Akhir dari kategori '{category}'.")
                    break

                response.raise_for_status()
                soup = BeautifulSoup(response.content, "html.parser")
                
                selector = "div[class*='article_recent__'] div[class*='article_recent_desc__'] h1 a"
                link_tags = soup.select(selector)
                
                if not link_tags:
                    log_to_file(f"    -> Tidak ada artikel ditemukan. Akhir dari kategori '{category}'.")
                    break
                
                page_urls = [tag['href'] for tag in link_tags if tag.has_attr('href')]
                
                log_to_file(f"    -> Ditemukan {len(page_urls)} link artikel.")
                
                with open(URL_OUTPUT_FILE, 'a', encoding='utf-8') as f:
                    for url in page_urls:
                        f.write(f"Indogamers;{url}\n")
                
                urls_per_category += len(page_urls)
                page_num += 1 
                
                time.sleep(1) 
                
            except requests.exceptions.RequestException as e:
                log_to_file(f"    -> ERROR: Gagal mengambil {page_url}: {e}. Mencoba lagi dalam 5 detik...")
                time.sleep(5)
                continue 
        
        log_to_file(f"Selesai kategori '{category}'. Total link ditemukan: {urls_per_category}")
        total_urls_found_session += urls_per_category

    log_to_file(f"Total link yang didapat pada sesi ini: {total_urls_found_session}")
    log_to_file("===== Proses Crawling Selesai =====\n")

if __name__ == "__main__":
    crawl_indogamers_deep()

[2025-11-09 02:43:14] ===== Memulai Proses Crawling Mendalam Indogamers.com =====
[2025-11-09 02:43:14] Memulai kategori: 'e-sport'
[2025-11-09 02:43:14]   -> Memproses halaman: https://indogamers.com/e-sport?page=1
[2025-11-09 02:43:15]     -> Ditemukan 20 link artikel.
[2025-11-09 02:43:16]   -> Memproses halaman: https://indogamers.com/e-sport?page=2
[2025-11-09 02:43:16]     -> Ditemukan 20 link artikel.
[2025-11-09 02:43:17]   -> Memproses halaman: https://indogamers.com/e-sport?page=3
[2025-11-09 02:43:18]     -> Ditemukan 20 link artikel.
[2025-11-09 02:43:19]   -> Memproses halaman: https://indogamers.com/e-sport?page=4
[2025-11-09 02:43:20]     -> Ditemukan 20 link artikel.
[2025-11-09 02:43:21]   -> Memproses halaman: https://indogamers.com/e-sport?page=5
[2025-11-09 02:43:21]     -> Ditemukan 20 link artikel.
[2025-11-09 02:43:22]   -> Memproses halaman: https://indogamers.com/e-sport?page=6
[2025-11-09 02:43:22]     -> Ditemukan 20 link artikel.
[2025-11-09 02:43:23]   -> M

### JagatPlay

In [10]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os
import re 
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# --- Konfigurasi ---
SITEMAP_INDEX_URL = "https://jagatplay.com/sitemap.html"
URL_OUTPUT_FILE = "../data/crawled_urls.txt"
LOG_FILE = "../data/crawl_logs.txt"
PORTAL_NAME = "Jagatplay"

output_dir = os.path.dirname(URL_OUTPUT_FILE)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def log_to_file(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] {message}\n"
    print(log_entry.strip())
    with open(LOG_FILE, 'a', encoding='utf-8') as f:
        f.write(log_entry)

def get_jagatplay_sitemap_number(url):
    """Mengekstrak nomor dari URL sitemap untuk sorting yang benar."""
    match = re.search(r'post-sitemap(\d+)\.html', url)
    return int(match.group(1)) if match else 0

def crawl_jagatplay():
    log_to_file(f"===== Memulai Proses Crawling {PORTAL_NAME} =====")
    
    all_urls_found = []
    
    try:
        log_to_file(f"Mengambil sitemap index dari: {SITEMAP_INDEX_URL}")
        response_main = requests.get(SITEMAP_INDEX_URL, headers=HEADERS, timeout=15, verify=False)
        response_main.raise_for_status()
        soup_main = BeautifulSoup(response_main.content, "html.parser")
        
        sitemap_tags = soup_main.select('tr > td > a')
        all_sitemap_links = [tag['href'] for tag in sitemap_tags if tag.has_attr('href')]
        
        if not all_sitemap_links:
            log_to_file("ERROR: Tidak ada link sitemap yang ditemukan di halaman index.")
            return

        log_to_file(f"Total sitemap ditemukan di index: {len(all_sitemap_links)}")
        
        post_sitemap_links_numbered = [url for url in all_sitemap_links if re.search(r'post-sitemap\d+\.html', url)]
        
        post_sitemap_links_numbered.sort(key=get_jagatplay_sitemap_number)
        
        sitemap_limit = 11
        sitemaps_to_process = [
            url for url in post_sitemap_links_numbered 
            if get_jagatplay_sitemap_number(url) <= sitemap_limit
        ]
        
        log_to_file(f"Menyaring sitemap... Akan memproses {len(sitemaps_to_process)} sitemap (hingga post-sitemap{sitemap_limit}).")

        for i, sitemap_url in enumerate(sitemaps_to_process, 1):
            log_to_file(f"({i}/{len(sitemaps_to_process)}) Memproses sitemap: {sitemap_url}")
            try:
                response_post = requests.get(sitemap_url, headers=HEADERS, timeout=15, verify=False)
                response_post.raise_for_status()

                soup_post = BeautifulSoup(response_post.content, "xml")
                
                article_links = []
                url_blocks = soup_post.find_all('url')
                for block in url_blocks:
                    loc_tag = block.find('loc')
                    if loc_tag:
                        article_links.append(loc_tag.text)
                
                log_to_file(f"  -> Ditemukan {len(article_links)} link artikel.")
                all_urls_found.extend(article_links)

            except requests.exceptions.RequestException as e:
                log_to_file(f"  -> ERROR: Gagal mengambil atau memproses {sitemap_url}: {e}")
                continue

    except requests.exceptions.RequestException as e:
        log_to_file(f"FATAL ERROR: Gagal mengambil sitemap index utama. Proses dihentikan. Error: {e}")
        return

    if all_urls_found:
        unique_urls = sorted(list(set(all_urls_found)))
        log_to_file(f"Menyimpan total {len(unique_urls)} link unik ke file {URL_OUTPUT_FILE}...")
        try:
            with open(URL_OUTPUT_FILE, 'a', encoding='utf-8') as f:
                for url in unique_urls:
                    f.write(f"{PORTAL_NAME};{url}\n")
            log_to_file("Penyimpanan link berhasil.")
        except IOError as e:
            log_to_file(f"ERROR: Gagal menulis ke file {URL_OUTPUT_FILE}: {e}")
    else:
        log_to_file("Tidak ada link baru yang ditemukan pada sesi crawling ini.")
    
    log_to_file(f"Total link yang didapat pada sesi ini: {len(all_urls_found)}")
    log_to_file("===== Proses Crawling Selesai =====\n")


if __name__ == "__main__":
    crawl_jagatplay()

[2025-11-09 02:56:06] ===== Memulai Proses Crawling Jagatplay =====
[2025-11-09 02:56:06] Mengambil sitemap index dari: https://jagatplay.com/sitemap.html
[2025-11-09 02:56:08] Total sitemap ditemukan di index: 24
[2025-11-09 02:56:08] Menyaring sitemap... Akan memproses 10 sitemap (hingga post-sitemap11).
[2025-11-09 02:56:08] (1/10) Memproses sitemap: https://jagatplay.com/post-sitemap2.html
[2025-11-09 02:56:11]   -> Ditemukan 1000 link artikel.
[2025-11-09 02:56:11] (2/10) Memproses sitemap: https://jagatplay.com/post-sitemap3.html
[2025-11-09 02:56:14]   -> Ditemukan 1000 link artikel.
[2025-11-09 02:56:14] (3/10) Memproses sitemap: https://jagatplay.com/post-sitemap4.html
[2025-11-09 02:56:18]   -> Ditemukan 1000 link artikel.
[2025-11-09 02:56:18] (4/10) Memproses sitemap: https://jagatplay.com/post-sitemap5.html
[2025-11-09 02:56:21]   -> Ditemukan 1000 link artikel.
[2025-11-09 02:56:21] (5/10) Memproses sitemap: https://jagatplay.com/post-sitemap6.html
[2025-11-09 02:56:24]  

## **Scraping**

In [14]:
import requests
from bs4 import BeautifulSoup, NavigableString, Comment
from datetime import datetime
import pandas as pd
import os
import time
import re
import urllib3

# Nonaktifkan pesan peringatan SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# --- Konfigurasi ---
CRAWLED_URL_FILE = "../data/crawled_urls.txt"
OUTPUT_CSV_FILE = "../data/scraped_articles.csv"
LOG_FILE = "../data/scrape_logs.txt"

PORTALS_TO_SCRAPE = [
    "Detik-Inet",
    "Gamebrott",
    "Kotakgame",
    "Indogamers",
    "Jagatplay"
]

os.makedirs(os.path.dirname(OUTPUT_CSV_FILE), exist_ok=True)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def log_to_file(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] {message}\n"
    print(log_entry.strip())
    with open(LOG_FILE, 'a', encoding='utf-8') as f:
        f.write(log_entry)

def read_all_urls_to_scrape(filepath, portal_names):
    log_to_file(f"Membaca SEMUA URL dari {filepath}...")
    all_tasks = []
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                if ';' in line:
                    portal, url = line.strip().split(';', 1)
                    if portal in portal_names:
                        all_tasks.append((portal, url))
        log_to_file(f"Total URL target ditemukan: {len(all_tasks)}")
        return all_tasks
    except FileNotFoundError:
        log_to_file(f"ERROR: File {filepath} tidak ditemukan.")
        return []

def get_already_scraped_urls(filepath):
    """Membaca CSV dan mengembalikan set URL yang sudah di-scrape."""
    if not os.path.exists(filepath):
        return set()
    try:
        df = pd.read_csv(filepath)
        if 'url' in df.columns:
            return set(df['url'])
        else:
            log_to_file("WARNING: Kolom 'url' tidak ditemukan di CSV. Tidak bisa melanjutkan. Harap hapus file CSV lama atau tambahkan kolom 'url'.")
            # Mengembalikan set kosong agar skrip bisa berjalan (tapi akan menimpa) atau hentikan paksa
            return set()
    except pd.errors.EmptyDataError:
        return set() # File ada tapi kosong
    except Exception as e:
        log_to_file(f"Error saat membaca CSV yang ada: {e}. Mengasumsikan file kosong.")
        return set()

# (Semua fungsi scraper individu tidak berubah)
def scrape_gamebrott_article(url, soup):
    title_tag = soup.select_one('h1.jeg_post_title')
    title = title_tag.get_text(strip=True) if title_tag else 'N/A'
    thumb_tag = soup.select_one('.jeg_featured.featured_image .thumbnail-container img')
    thumbnail_url = thumb_tag['src'] if thumb_tag and thumb_tag.has_attr('src') else 'N/A'
    date_tag = soup.select_one('.jeg_meta_date a')
    publish_date = date_tag.get_text(strip=True) if date_tag else 'N/A'
    content_div = soup.select_one('.content-inner')
    if content_div:
        paragraphs = content_div.find_all('p')
        content = '\n'.join([p.get_text(strip=True) for p in paragraphs])
    else: content = 'N/A'
    return {"judul": title, "konten": content, "tanggal_terbit": publish_date, "url_thumbnail": thumbnail_url}
def scrape_kotakgame_article(url, soup):
    title_tag = soup.select_one('h3.judulh3 span.txt9')
    title = title_tag.get_text(strip=True) if title_tag else 'N/A'
    thumb_tag = soup.select_one('div.wrapimg img')
    if thumb_tag and thumb_tag.has_attr('src'):
        relative_url = thumb_tag['src']
        thumbnail_url = f"https://www.kotakgame.com{relative_url}" if relative_url.startswith('/') else relative_url
    else: thumbnail_url = 'N/A'
    date_span = soup.select_one('.boxcreate .txtcreate2')
    if date_span:
        full_text = date_span.get_text(strip=True)
        try:
            match = re.search(r'\|\s*(.*)', full_text)
            publish_date = match.group(1).strip() if match else 'N/A'
        except (AttributeError, IndexError): publish_date = full_text
    else: publish_date = 'N/A'
    content_div = soup.select_one('div.isinewsp')
    content = ""
    if content_div:
        for blockquote in content_div.find_all('blockquote'): blockquote.decompose()
        for insta_block in content_div.find_all(text=re.compile("View this post on Instagram")):
            if insta_block.parent.name in ['div', 'p', 'span']: insta_block.parent.decompose()
        content_full = content_div.get_text(separator='\n', strip=True)
        content_lines = []
        for line in content_full.split('\n'):
            if "Baca ini juga" in line or "Selain berita utama di atas" in line: break
            content_lines.append(line)
        content = '\n'.join(content_lines)
    if not content: content = 'N/A'
    return {"judul": title, "konten": content.strip(), "tanggal_terbit": publish_date, "url_thumbnail": thumbnail_url}
def scrape_indogamers_article(url, soup):
    title_tag = soup.select_one('h1[class*="style_article__title__"]')
    title = title_tag.get_text(strip=True) if title_tag else 'N/A'
    thumb_tag = soup.select_one('div[class*="style_image__article__"] img')
    if thumb_tag and thumb_tag.has_attr('srcset'):
        last_url_part = thumb_tag['srcset'].split(',')[-1].strip()
        relative_url = last_url_part.split(' ')[0]
        thumbnail_url = f"https://indogamers.com{relative_url}"
    else: thumbnail_url = 'N/A'
    date_container = soup.select_one('div[class*="style_author__box__"]')
    publish_date = 'N/A'
    if date_container:
        all_spans = date_container.find_all('span')
        for span in all_spans:
            span_text = span.get_text(strip=True)
            if re.search(r'\b(Senin|Selasa|Rabu|Kamis|Jumat|Sabtu|Minggu)\b', span_text):
                publish_date = span_text.split(',')[0].strip()
                break
    content_div = soup.find('article')
    content = 'N/A'
    if content_div:
        paragraphs = content_div.find_all('p')
        content = '\n'.join([p.get_text(strip=True) for p in paragraphs if not p.has_attr('class') or 'caption' not in ''.join(p['class'])])
    return {"judul": title, "konten": content, "tanggal_terbit": publish_date, "url_thumbnail": thumbnail_url}
def scrape_jagatplay_article(url, soup):
    title = 'N/A'; thumbnail_url = 'N/A'; publish_date = 'N/A'; content = 'N/A'
    title_tag = soup.select_one('div.jgpost__header h1')
    if title_tag: title = title_tag.get_text(strip=True)
    thumb_div = soup.select_one('div.jgpost__feat-img')
    if thumb_div and thumb_div.has_attr('style'):
        match = re.search(r"url\(['\"]?(.*?)['\"]?\)", thumb_div['style'])
        if match: thumbnail_url = match.group(1)
    date_container = soup.select_one('div.jgauthor__posted')
    if date_container:
        child_divs = date_container.find_all('div', recursive=False)
        for div in child_divs:
            if not div.find('a'):
                publish_date = div.get_text(strip=True)
                break
    content_container = soup.select_one('div.jgpost__content')
    if content_container:
        post_content_comment = content_container.find(string=lambda text: isinstance(text, Comment) and "Post Content" in text)
        if post_content_comment:
            content_list = []
            for sibling in post_content_comment.find_next_siblings():
                if sibling.name == 'div' and ('iklan-inline1' in sibling.get('class', []) or 'heateor_sss_sharing_container' in sibling.get('class', [])): break
                if sibling.name == 'p': content_list.append(sibling.get_text(strip=True))
            if content_list: content = '\n'.join(filter(None, content_list))
    return {"judul": title, "konten": content.strip(), "tanggal_terbit": publish_date, "url_thumbnail": thumbnail_url}
def scrape_detikinet_article(url, soup):
    title = 'N/A'; thumbnail_url = 'N/A'; publish_date = 'N/A'; content = 'N/A'
    title_tag = soup.select_one('h1.detail__title')
    if title_tag: title = title_tag.get_text(strip=True)
    date_tag = soup.select_one('div.detail__date')
    if date_tag:
        full_date_text = date_tag.get_text(strip=True)
        try:
            date_part = full_date_text.split(', ')[1]
            publish_date = date_part.split(' WIB')[0].strip()
        except IndexError: publish_date = full_date_text
    thumb_tag = soup.select_one('figure.detail__media-image img')
    if thumb_tag and thumb_tag.has_attr('src'): thumbnail_url = thumb_tag['src']
    content_div = soup.select_one('div#detikdetailtext')
    if content_div:
        elements_to_remove_selectors = ['.collapsible', '.parallaxindetail', '.noncontent', '.staticdetail_container', '.aevp', 'script', 'style', 'table.pic_artikel_sisip_table']
        for selector in elements_to_remove_selectors:
            for element in content_div.select(selector): element.decompose()
        last_strong = content_div.find_all('strong')
        if last_strong and re.match(r'\(\w+/\w+\)', last_strong[-1].get_text(strip=True)): last_strong[-1].decompose()
        paragraphs = content_div.find_all('p')
        content_list = [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)]
        if content_list: content = '\n'.join(content_list)
    return {"judul": title, "konten": content, "tanggal_terbit": publish_date, "url_thumbnail": thumbnail_url}

def main():
    log_to_file("===== Memulai Proses Scraping Skala Penuh (Mode Resume + Real-time Save) =====")
    
    all_tasks = read_all_urls_to_scrape(CRAWLED_URL_FILE, PORTALS_TO_SCRAPE)
    
    header = ['id_dokumen', 'sumber', 'url', 'judul', 'konten', 'tanggal_terbit', 'url_thumbnail']
    
    already_scraped_urls = get_already_scraped_urls(OUTPUT_CSV_FILE)
    if already_scraped_urls:
        log_to_file(f"Ditemukan {len(already_scraped_urls)} URL yang sudah diproses. Akan melanjutkan.")
    
    doc_id_counter = len(already_scraped_urls) + 1
    
    if not os.path.exists(OUTPUT_CSV_FILE) or not already_scraped_urls:
        log_to_file(f"File {OUTPUT_CSV_FILE} tidak ada atau kosong. Membuat file baru dengan header.")
        pd.DataFrame(columns=header).to_csv(OUTPUT_CSV_FILE, index=False)
        doc_id_counter = 1 # Pastikan counter disetel ulang jika file baru
    
    total_urls_to_process = len(all_tasks)
    newly_scraped_count = 0
    
    for i, (portal, url) in enumerate(all_tasks):
        # Lewati URL yang sudah ada
        if url in already_scraped_urls:
            continue
            
        log_to_file(f"  ({i+1}/{total_urls_to_process}) Scraping: {url}")
        
        try:
            response = requests.get(url, headers=HEADERS, timeout=15, verify=False)
            if response.status_code != 200:
                log_to_file(f"    -> Gagal mengakses (Status: {response.status_code})")
                continue
            soup = BeautifulSoup(response.content, 'html.parser')

            data = None
            if portal == "Gamebrott": data = scrape_gamebrott_article(url, soup)
            elif portal == "Kotakgame": data = scrape_kotakgame_article(url, soup)
            elif portal == "Indogamers": data = scrape_indogamers_article(url, soup)
            elif portal == "Jagatplay": data = scrape_jagatplay_article(url, soup)
            elif portal == "Detik-Inet": data = scrape_detikinet_article(url, soup)
            
            if data:
                data['id_dokumen'] = f"doc_{doc_id_counter:05d}"
                data['sumber'] = portal
                data['url'] = url
                
                df_row = pd.DataFrame([data])
                df_row = df_row[header]
                
                df_row.to_csv(OUTPUT_CSV_FILE, mode='a', index=False, header=False)
                
                doc_id_counter += 1
                newly_scraped_count += 1

            time.sleep(0.5)

        except Exception as e:
            log_to_file(f"    -> Terjadi error kritis saat scraping {url}: {e}")

    log_to_file(f"Scraping selesai. Total {newly_scraped_count} artikel BARU berhasil disimpan ke {OUTPUT_CSV_FILE}.")
    log_to_file(f"Proses selesai.\n")

if __name__ == "__main__":
    main()

[2025-11-09 20:09:29] ===== Memulai Proses Scraping Skala Penuh (Mode Resume + Real-time Save) =====
[2025-11-09 20:09:29] Membaca SEMUA URL dari ../data/crawled_urls.txt...
[2025-11-09 20:09:29] Total URL target ditemukan: 50658
[2025-11-09 20:09:29] Ditemukan 39 URL yang sudah diproses. Akan melanjutkan.
[2025-11-09 20:09:29]   (40/50658) Scraping: https://inet.detik.com/games-news/d-4876363/menebak-bayaran-ninja-saat-dibajak-microsoft-dari-twitch
[2025-11-09 20:09:29]   (41/50658) Scraping: https://inet.detik.com/games-news/d-4876418/server-pubg-jebol-gegara-wabah-virus-corona
[2025-11-09 20:09:30]   (42/50658) Scraping: https://inet.detik.com/games-news/d-4877060/virus-corona-dan-cocoklogi-game-resident-evil
[2025-11-09 20:09:31]   (43/50658) Scraping: https://inet.detik.com/games-news/d-4878712/iegs-2020-diramaikan-diskon-pc-dan-berbagai-turnamen-game
[2025-11-09 20:09:32]   (44/50658) Scraping: https://inet.detik.com/games-news/d-4879190/turnamen-overwatch-league-di-china-dibatal

KeyboardInterrupt: 

## **Preprocessing**